In [ ]:
%pip install requests
%pip install pandas
%pip install -U scikit-learn

In [3]:
import csv
import json
import os
import pandas as pd
from tqdm.notebook import tqdm
from ast import literal_eval
import math
from concurrent.futures import ThreadPoolExecutor
import requests
import random
import subprocess

import os
import numpy as np
import pandas as pd
import torch

In [4]:
ROOT_DIR = os.path.abspath("")
DATA_DIR = os.path.join(ROOT_DIR, 'CrabNet/data/materials_data')
TRAINING_CSV_FILE = os.path.join(ROOT_DIR, "Vacancyenergydata.csv")
API_KEY = "3XX6x9Nd3cKWf9Q4wpaio64CBHFp2uDq"
TRAINING_DATA = []

print(TRAINING_CSV_FILE)

d:\Projects and Research\Python Projects\Kevin-ML-perovskites-project\Vacancyenergydata.csv


## Step1:Collecting data.

#### Load Training and Prediction data from the csv file.

In [5]:
df = pd.read_csv(TRAINING_CSV_FILE)

print(df.columns)
print(df.head())

df = df[['Chemical formula', 'Vacancy energy [eV/O atom]']]
new_names = {
    'Chemical formula': 'chem_formula',
    'Vacancy energy [eV/O atom]': 'e_ovac'
}
df.rename(columns=new_names, inplace=True)
df = df.to_dict("records")
print(df[:10])


Index(['Chemical formula', 'A', 'B', 'In literature', 'Valence A', 'Valence B',
       'Radius A [ang]', 'Radius B [ang]', 'Lowest distortion',
       'Formation energy [eV/atom]', 'Stability [eV/atom]',
       'Magnetic moment [mu_B]', 'Volume per atom [A^3/atom]', 'Band gap [eV]',
       'a [ang]', 'b [ang]', 'c [ang]', 'alpha [deg]', 'beta [deg]',
       'gamma [deg]', 'Vacancy energy [eV/O atom]'],
      dtype='object')
  Chemical formula   A   B  In literature          Valence A  \
0            Ac2O3  Ac  Ac          False  element not in BV   
1           AcAgO3  Ac  Ag          False  element not in BV   
2           AcAlO3  Ac  Al          False  element not in BV   
3           AcAsO3  Ac  As          False  element not in BV   
4           AcAuO3  Ac  Au          False  element not in BV   

           Valence B  Radius A [ang]  Radius B [ang] Lowest distortion  \
0  element not in BV            1.12            1.12             cubic   
1  element not in BV            1.12   

## Step 2: Data cleaning

#### Cleaning training data

In [6]:
TRAINING_DATA = [m for m in df if m['e_ovac'] != '-']
PREDICTION_DATA = [m for m in df if m['e_ovac'] == '-']

print("Materials in the csv file without E_Ovac: ", len(PREDICTION_DATA))
print("Materials in the csv file with E_Ovac: ", len(TRAINING_DATA))
print("Total materials in the csv file: ", len(df))


Materials in the csv file without E_Ovac:  415
Materials in the csv file with E_Ovac:  4914
Total materials in the csv file:  5329


In [7]:
print(TRAINING_DATA[0])

{'chem_formula': 'Ac2O3', 'e_ovac': '3.150'}


In [8]:
print(PREDICTION_DATA[0])

{'chem_formula': 'AcMgO3', 'e_ovac': '-'}


### Create Model Training Data (Create train.csv, val.csv, test.csv)

##### 1) Split the data into train, val and test sets as required by the Crabnet model.

In [9]:
## Split the data into train, test, val

random.shuffle(TRAINING_DATA) # Shuffle the training data list on every run
split_ratio = "6:2:2" # Set the ratio at with to split the data into train, test and validation sets from the training data list

ratios = [round(int(r)/10 * len(TRAINING_DATA)) for r in split_ratio.split(':')] # Get the number of materials from the training data list according tp the ratio

# Get X amount of data from the training dataset accoring to the set rratio
train_set = TRAINING_DATA[:ratios[0]] 
test_set = TRAINING_DATA[ratios[0]:(ratios[0] + ratios[1])]
val_set = TRAINING_DATA[(ratios[0] + ratios[1]):]

print(len(train_set), len(test_set), len(val_set))
print(ratios, sum(ratios), len(TRAINING_DATA))

print(train_set[:5])
print(test_set[:5])
print(val_set[:5])
# print(TRAINING_DATA[:5])

2948 983 983
[2948, 983, 983] 4914 4914
[{'chem_formula': 'RbPtO3', 'e_ovac': '-0.383'}, {'chem_formula': 'ThAuO3', 'e_ovac': '-1.250'}, {'chem_formula': 'TePbO3', 'e_ovac': '0.913'}, {'chem_formula': 'AuScO3', 'e_ovac': '-3.267'}, {'chem_formula': 'WGeO3', 'e_ovac': '0.254'}]
[{'chem_formula': 'AcUO3', 'e_ovac': '2.892'}, {'chem_formula': 'HoGaO3', 'e_ovac': '4.388'}, {'chem_formula': 'PdAuO3', 'e_ovac': '-1.681'}, {'chem_formula': 'InPrO3', 'e_ovac': '-4.900'}, {'chem_formula': 'AsZrO3', 'e_ovac': '-0.708'}]
[{'chem_formula': 'RuNbO3', 'e_ovac': '3.675'}, {'chem_formula': 'SiHgO3', 'e_ovac': '-7.102'}, {'chem_formula': 'LuCrO3', 'e_ovac': '3.926'}, {'chem_formula': 'HfOsO3', 'e_ovac': '-2.885'}, {'chem_formula': 'RuDyO3', 'e_ovac': '-6.081'}]


##### 2) Write the data to csv files

In [10]:
## Save data to CSV

fields = ['formula', 'target']
trin_rows = [[data.get('chem_formula'), data.get('e_ovac')] for data in train_set]
val_rows = [[data.get('chem_formula'), data.get('e_ovac')] for data in val_set]
test_rows = [[data.get('chem_formula'), data.get('e_ovac')] for data in test_set]

# print(rows)
## Create train.csv with train_set data
with open(os.path.join(DATA_DIR, 'train.csv'), 'w') as train_file:
    csvwriter = csv.writer(train_file)
    
    csvwriter.writerow(fields)
    csvwriter.writerows(trin_rows)

## Create val.csv with train_set data
with open(os.path.join(DATA_DIR, 'val.csv'), 'w') as val_file:
    csvwriter = csv.writer(val_file)
    
    csvwriter.writerow(fields)
    csvwriter.writerows(val_rows)

## Create test.csv with train_set data
with open(os.path.join(DATA_DIR, 'test.csv'), 'w') as test_file:
    csvwriter = csv.writer(test_file)
    
    csvwriter.writerow(fields)
    csvwriter.writerows(test_rows)
    

#### Train the crabnet model

In [11]:

train_script = os.path.join(ROOT_DIR, 'CrabNet/train_crabnet.py')
os.chdir(os.path.join(ROOT_DIR, 'CrabNet'))

command = f"python \"{train_script}\""
print(f"RUNNING COMMAND: {command}")

command = "python -m pip freeze"
# Run the script showing the output of stdout and stderr line by line
proc = subprocess.Popen(
    command, 
    shell=True, 
    stdout=subprocess.PIPE, 
    stderr=subprocess.PIPE,
)

output, errors = proc.communicate()
print(f"OUTPUT >> {output}")
print(f"ERRORS >> {errors}")



RUNNING COMMAND: python "d:\Projects and Research\Python Projects\Kevin-ML-perovskites-project\CrabNet/train_crabnet.py"

Property "example_materials_property" selected for training

Model architecture: out_dims, d_model, N, heads
3, 512, 3, 4
Running on compute device: cpu
Model size: 11987206 parameters

Generating EDM: 100%|█████████████████████████████████████████████████████| 3393/3393 [00:00<00:00, 72234.83formulae/s]
loading data with up to 3 elements in the formula
training with batchsize 256 (2**8.000)
Generating EDM: 100%|███████████████████████████████████████████████████████| 727/727 [00:00<00:00, 55941.49formulae/s]
loading data with up to 3 elements in the formula
stepping every 14 training passes, cycling lr every 1 epochs
checkin at 2 epochs to match lr scheduler
Epoch: 0/40 --- train mae: 25.5 val mae: 25.7
Epoch: 1/40 --- train mae: 26.5 val mae: 27.2
Epoch: 3/40 --- train mae: 21 val mae: 22
Epoch: 5/40 --- train mae: 15 val mae: 15.8
Epoch: 7/40 --- train mae: 13.3 